# LLM Prompt Tuning Project

### Sentiment Analysis on Financial Text

The goal of this check in is to fully implement your desired training approach for your task and evaluate pre and post training performance on your full evaluation benchmark datasets. This notebook will guide you through the necessary steps. 

**References**

1. model: https://huggingface.co/meta-llama/Llama-3.2-3B-Instruct
2. financial dataset: https://huggingface.co/datasets/warwickai/financial_phrasebank_mirror
3. 1 benchmark (multifin): https://huggingface.co/datasets/ChanceFocus/flare-multifin-en/viewer?row=17&views%5B%5D=test
4. Two of the benchmarks: https://github.com/EleutherAI/lm-evaluation-harness/tree/main/lm_eval/tasks
5. Prompt engineering: https://www.matillion.com/blog/large-language-model-prompt-engineering-for-common-data-problems
6. Prompt engineering: https://medium.com/towards-data-science/in-context-learning-approaches-in-large-language-models-9c0c53b116a1
7. Prompt engineering: previous project checkin
8. Prompt tuning: previous homework and notebooks from course lecture

## Step 1: Choose the Training Approach


**ANSWER**

I will be implementing prompt tuning on the GPT2-instruct model. Prompt tuning typically works well for text classification texts, which applies to my use case because I have 3 sentiment categories the model must choose from. The financial phrasebank should lend well to prompt tuning because it includes slightly less than 4,000 labeled training observations, which should be sufficient for prompt tuning. The 4,000 may be on the lower end for what is needed, so I may need to try LoRA to actually modify model weights. 

I am not making this decision because of the empirical evidence from prompt tuning in the previous homework. I used an unrelated dataset in the assignment, and as I concluded in the homework, this dataset was insufficient and for an improper task - math problems. I would like to attempt prompt tuning before using LoRA or finetuning because I think it would suit the task well and is quicker. I will combine prompt tuning with the prompt engineering I previously used - this implements few-shot prompting. I decided to use GPT2 instruct after attempting to benchmark on llama3-3B instruct, which would not run with the current computing resources. The drawback of using prompt tuning on GPT2-instruct is that the model / training-method combination may be insufficient to not only shape the information the model pulls from in its response but the response itself may be formatted incorrectly. In previous project check-ins, I found that using the message format with llama 3-3B instruct returned properly formatted and concise responses. I will address by formatting the model response myself, if it is not a one-word response as desired.

## Step 2: Benchmark the Model


### Import model and dependencies

In [1]:
import torch
import os
import pandas as pd
import numpy as np
import re
import time
from sklearn.model_selection import train_test_split
import lm_eval
import json
# Set the environment variable
os.environ['HF_HOME'] = '/scratch/dcc7qe/models/cache'

# Get the path from the environment variable
hf_home_path = os.environ['HF_HOME']

# Create the directory if it doesn't exist
os.makedirs(hf_home_path, exist_ok=True)
#NOTE- you must set the directory for HF_HOME before importing anything for Transformers or it will default to downloading models in your /Home directory

import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM,setup_chat_format
from datasets import load_dataset, Dataset

/scratch/dcc7qe/llm_course/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
### set directories
#os.getcwd()

def create_dirs(model_name):
    '''
    Creates different folders for different models from huggingface - gpt or llama
    '''
    base_dir = '/sfs/gpfs/tardis/home/dcc7qe/LLM_project'
    
    model_name_dir = base_dir + f'/{model_name}_instruct'
    pre_results_dir = model_name_dir + '/pretrain_results'
    post_results_dir = model_name_dir + '/posttrain_results'
    trained_model_dir = model_name_dir + '/trained_models'
    best_model_results_dir = trained_model_dir + '/best_model'
    
    for new_dir in [model_name_dir, pre_results_dir, post_results_dir,
                   trained_model_dir, best_model_results_dir]:
        # create a directory
        if not os.path.exists(new_dir):
            os.mkdir(new_dir) 

    return(model_name_dir, pre_results_dir, post_results_dir,
           trained_model_dir, best_model_results_dir)

In [3]:
model_name_dir, pre_results_dir, post_results_dir, trained_model_dir, best_model_results_dir = create_dirs('gpt2')

In [4]:
def import_model_token(choice):
    '''
    Imports the model and tokenizer for the desired model
    '''
    tokenizer = AutoTokenizer.from_pretrained(choice)
    model = AutoModelForCausalLM.from_pretrained(choice)
    
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = tokenizer.pad_token_id
    
    print(model)
    return(model, tokenizer)

In [5]:
### choose between llama3 and gpt2
model_choice = "vicgalle/gpt2-open-instruct-v1" # used throughout the notebook
#model_choice = "meta-llama/Llama-3.2-3B-Instruct"

model, tokenizer = import_model_token(model_choice)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50260, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50260, bias=False)
)


### Import and prepare the financial-phrasebank dataset

#### 2a. Download and load into pandas. 

Create new column called "Instruction". 

In [6]:
finance_phrase = load_dataset("warwickai/financial_phrasebank_mirror")
finance_phrase = pd.DataFrame(finance_phrase['train']) # this is the only split of the dataset on huggingface.
finance_phrase

,sentence,label
0,"According to Gran , the company has no plans t...",1
1,Technopolis plans to develop in stages an area...,1
2,The international electronic industry company ...,0
3,With the new production plant the company woul...,2
4,According to the company 's updated strategy f...,2
...,...,...
4841,LONDON MarketWatch -- Share prices ended lower...,0
4842,Rinkuskiai 's beer sales fell by 6.5 per cent ...,1
4843,Operating profit fell to EUR 35.4 mn from EUR ...,0
4844,Net sales of the Paper segment decreased to EU...,0


In [7]:
# split the data
train, test = train_test_split(finance_phrase, test_size=0.2)
print(train.shape)
print(test.shape)

(3876, 2)
(970, 2)


In [8]:
train['Instruction'] = train['sentence']
test['Instruction'] = test['sentence']

#### 2b. Create column called "response"

In [9]:
train['Response'] = train['label']
for new, old in {'negative': 0, 'neutral': 1, 'positive': 2}.items():
    train.loc[train['Response'] == old, ['Response']] = new 
    
test['Response'] = test['label']
for new, old in {'negative': 0, 'neutral': 1, 'positive': 2}.items():
    test.loc[test['Response'] == old, ['Response']] = new 

/tmp/ipykernel_538083/2595862958.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'negative' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  train.loc[train['Response'] == old, ['Response']] = new
/tmp/ipykernel_538083/2595862958.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'negative' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  test.loc[test['Response'] == old, ['Response']] = new


#### 2c. Set aside 9 examples for few shot prompting

In [10]:
train_temp = pd.DataFrame()
few_shot_fin = pd.DataFrame()
for cat in list(train['Response'].unique()):
    temp = train[train['Response'] == cat]
    few_shot_fin = pd.concat([few_shot_fin, temp[-3:]])
    train_temp = pd.concat([train_temp, temp[:-3]])
print(few_shot_fin.shape)
train = train_temp
print(train.shape)

(9, 4)
(3867, 4)


In [11]:
few_shot_fin

,sentence,label,Instruction,Response
3023,Market data and analytics are derived from pri...,1,Market data and analytics are derived from pri...,neutral
3466,- Cash flow from operating activities before i...,1,- Cash flow from operating activities before i...,neutral
1983,The dollar bounced back after hitting another ...,1,The dollar bounced back after hitting another ...,neutral
132,The major breweries increased their domestic b...,2,The major breweries increased their domestic b...,positive
1915,The Commission is to be applauded for applying...,2,The Commission is to be applauded for applying...,positive
220,"In the third quarter , net sales increased by ...",2,"In the third quarter , net sales increased by ...",positive
4664,Comparable operating profit decreased to EUR 1...,0,Comparable operating profit decreased to EUR 1...,negative
544,"However , the growth margin slowed down due to...",0,"However , the growth margin slowed down due to...",negative
1150,"According to Arokarhu , some of the purchases ...",0,"According to Arokarhu , some of the purchases ...",negative


#### 2d. Show the head

In [12]:
train.head()

,sentence,label,Instruction,Response
1531,Handelsbanken ranked before Local Cooperative ...,2,Handelsbanken ranked before Local Cooperative ...,positive
2260,Brazilian non-profit interbank Camara Interban...,2,Brazilian non-profit interbank Camara Interban...,positive
2272,CEO of the company Tarmo Noop said the growth ...,2,CEO of the company Tarmo Noop said the growth ...,positive
898,The company has established a 3G base station ...,2,The company has established a 3G base station ...,positive
1951,`` Indo-Russia trade can cross the targeted 10...,2,`` Indo-Russia trade can cross the targeted 10...,positive


#### 2e. Concatenate two different instruction-response pairs

In [13]:
print(list(train['Instruction'])[0] + '\n\n' + list(train['Instruction'])[1], "\n")

print(list(train['Response'])[0] + '\n\n' + list(train['Response'])[1])

Handelsbanken ranked before Local Cooperative Banks and Aktia in customer loyalty this time too , however .

Brazilian non-profit interbank Camara Interbancaria de Pagamentos CIP has acquired solutions from US business integration solutions provider Sterling Commerce , the latter company said in a statement . 

positive

positive


#### 2f. Add instruct column to the training and testing data

In [12]:
print(train.shape)
print(test.shape)

(3867, 4)
(970, 4)


In [13]:
train_copy = train.copy()
test_copy = test.copy()

In [14]:
train = train[['Instruction', 'Response']]
train.columns = ['question', 'answer']
test = test[['Instruction', 'Response']]
test.columns = ['question', 'answer']

In [15]:
# formats into the proper instruction/response format
options = 'positive, neutral, negative'

#### training data
question_list = list(train['question'])
answer_list = list(train['answer'])
instruct_list = []
question_formatted = []

for ind in range(len(question_list)):
    
    q = question_list[ind]
    #instruction = f'{q} Choose one of the following words: {options}'
    instruction = f'{q}'
    question_formatted.append(instruction)
    
    response = answer_list[ind]
        
    instruct = f"Instruct: {instruction}\n\nResponse: {response}"
    instruct_list.append(instruct)

# without reducing to only the needed columns, the tensors cannot be processed. 
train['instruct'] = instruct_list
train['question_formatted'] = question_formatted
train = train[['question_formatted', 'answer', 'instruct']]
train.columns = ['question', 'answer', 'instruct']

#### test data
question_list = list(test['question'])
answer_list = list(test['answer'])
instruct_list = []
question_formatted = []

for ind in range(len(question_list)):
    
    q = question_list[ind]
    instruction = f'{q} Choose one of the following words: {options}'
    question_formatted.append(instruction)
    
    response = answer_list[ind]
        
    instruct = f"Instruct: {instruction}\n\nResponse: {response}"
    instruct_list.append(instruct)

# without reducing to only the needed columns, the tensors cannot be processed. 
test['instruct'] = instruct_list
test['question_formatted'] = question_formatted
test = test[['question_formatted', 'answer', 'instruct']]
test.columns = ['question', 'answer', 'instruct']

/tmp/ipykernel_538083/3729892796.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['instruct'] = instruct_list


In [16]:
test.head(1)

,question,answer,instruct
1004,`` BG Crane has been a strong partner for Hiab...,neutral,Instruct: `` BG Crane has been a strong partne...


In [17]:
train.head(1)

,question,answer,instruct
3063,Okhta Center area is expected to have about 70...,neutral,Instruct: Okhta Center area is expected to hav...


### Query functions

In [18]:
standard_fin_sentiment_instruction = """You are a financial expert that analyzes financial news reports. Each report contains numbers and 
                                    qualitative descriptions of a subject's financial performance. 
                                    Your job is to classify the sentiment of each report as one of the predefined categories.
                                    Sentiment categories: ['positive', 'neutral', 'negative']. Reply with one word only.
                                    Following are a few examples of similar text and the correct category. 
                                    Provide the sentiment category for the last body of text."""

In [19]:
def get_prompt(question = '', instruction = False,
               few_shot_instruction = [], few_shot_response = []):
    '''
    Receives the few_shot example questions and responses. 
    Receives the prompt question and its label, if testing.
    '''
    
    if (instruction == False):
        prompt_instruction = standard_fin_sentiment_instruction
    else: 
        prompt_instruction = instruction # different instruction for multifin benchmark vs inference on the prompt-tuned model
        
    
    for ind in range(len(few_shot_instruction)):
        prompt_instruction = prompt_instruction + "\nText: {" + few_shot_instruction[ind] + "}\nAnswer: {" + few_shot_response[ind] + "}"
    
    prompt = prompt_instruction + "\nText: {" + question + "}\nAnswer:"
    return(prompt)

In [20]:
def get_pipe():
    pipe = pipeline("text-generation",
                    model=model,
                    tokenizer=tokenizer,
                    torch_dtype=torch.bfloat16,
                    device_map="auto")
    return(pipe)

In [21]:
def query_model(model_pipe, 
                question = '', instruction = False, 
                few_shot_instruction = [], few_shot_response = [],
                formatted_response = True
                ): # add parameters for testing vs inference?
    
    prompt = get_prompt(question, instruction, few_shot_instruction, few_shot_response)
    
    # queries the model via pipe
    sequences = model_pipe(prompt,
                           max_new_tokens=3, #### **important parameter - needs to == 3 for GPT2 instruct, for a concise response
                           do_sample=True,
                           top_k=10)
    
    if (formatted_response == True):
        formatted_response = format_response(sequences)
    else: formatted_response = ''
    
    return(sequences, formatted_response)

In [22]:
def format_response(output):
    ##### I need to modify this so that it finds the last instance of the 'answer' and captures that.
    
    text_raw = output[0]['generated_text'].split('\n')
    text_raw = [s for s in text_raw if (s.startswith('Text') or s.startswith('Answer'))]
    full_text = text_raw[-2:]
    
    answer = [s for s in full_text if (s.startswith('Answer'))][0]#.split(':')
    #answer = answer[answer.find('{'):]
    answer = answer[answer.find(':') + 1:]
    ############
    '''
    if (',' in answer):
        answer = answer.split(',')[:1][0]
    if (' ' in answer):
        answer = answer.split(' ')
    
    answer = [s for s in answer if (s != '')]
    if (len(answer) > 2): answer = answer[:1][0]
    else: answer = answer[0]
    answer = re.sub('[^a-zA-Z]', '', answer)
    #############
    '''
    answer = re.sub('[^a-zA-Z]', '', answer)
    answer = re.sub(' ', '', answer)
    
    
    formatted_response = {'full_text': full_text,
                          'model_answer': answer}
    
    return(formatted_response)

### Benchmarks: evaluate before training

#### Prepare custom benchmark

##### Benchmark data - multifin

In [79]:
multifin = load_dataset("ChanceFocus/flare-multifin-en")
multifin = pd.DataFrame(multifin['test']) # this is the only split of the dataset on huggingface.
multifin.head()

,id,query,answer,text,choices,gold
0,multifineng0,"In this task, you're working with English head...",Finance,Deal summary Helen Oy / Infratek Finland Oy,"[Finance, Technology, Tax & Accounting, Busine...",0
1,multifineng1,"In this task, you're working with English head...",Finance,Deals,"[Finance, Technology, Tax & Accounting, Busine...",0
2,multifineng2,"In this task, you're working with English head...",Technology,PwC named a Major Player in the IDC MarketScap...,"[Finance, Technology, Tax & Accounting, Busine...",1
3,multifineng3,"In this task, you're working with English head...",Tax & Accounting,Proposal for digital services tax,"[Finance, Technology, Tax & Accounting, Busine...",2
4,multifineng4,"In this task, you're working with English head...",Business & Management,Social Impact Lab,"[Finance, Technology, Tax & Accounting, Busine...",3


In [80]:
print(multifin.shape)
multifin['answer'].unique()

(546, 6)


array(['Finance', 'Technology', 'Tax & Accounting',
       'Business & Management', 'Government & Controls', 'Industry'],
      dtype=object)

In [81]:
few_shot_df = pd.DataFrame()
bench_df = pd.DataFrame()
for cat in list(multifin['answer'].unique()):
    temp = multifin[multifin['answer'] == cat]
    few_shot_df = pd.concat([few_shot_df, temp[-2:]])
    bench_df = pd.concat([bench_df, temp[:-2]])
print(few_shot_df.shape)
print(bench_df.shape)

(12, 6)
(534, 6)


In [82]:
few_shot_df

,id,query,answer,text,choices,gold
539,multifineng539,"In this task, you're working with English head...",Finance,PwC participated in the opening of the market ...,"[Finance, Technology, Tax & Accounting, Busine...",0
545,multifineng545,"In this task, you're working with English head...",Finance,Removal of the annual limit of the pension and...,"[Finance, Technology, Tax & Accounting, Busine...",0
520,multifineng520,"In this task, you're working with English head...",Technology,Technology has reduced the tax compliance burd...,"[Finance, Technology, Tax & Accounting, Busine...",1
544,multifineng544,"In this task, you're working with English head...",Technology,Information Security Strategy,"[Finance, Technology, Tax & Accounting, Busine...",1
542,multifineng542,"In this task, you're working with English head...",Tax & Accounting,Tax reporting & strategy,"[Finance, Technology, Tax & Accounting, Busine...",2
543,multifineng543,"In this task, you're working with English head...",Tax & Accounting,Taxes in Iceland,"[Finance, Technology, Tax & Accounting, Busine...",2
536,multifineng536,"In this task, you're working with English head...",Business & Management,The evolving nature of the green agenda,"[Finance, Technology, Tax & Accounting, Busine...",3
541,multifineng541,"In this task, you're working with English head...",Business & Management,Generational transfer,"[Finance, Technology, Tax & Accounting, Busine...",3
511,multifineng511,"In this task, you're working with English head...",Government & Controls,SOX and internal control,"[Finance, Technology, Tax & Accounting, Busine...",4
533,multifineng533,"In this task, you're working with English head...",Government & Controls,Government and Public Sector,"[Finance, Technology, Tax & Accounting, Busine...",4


In [83]:
few_shot_instruction = list(few_shot_df['text'])
few_shot_response = list(few_shot_df['answer'])
bench_questions = list(bench_df['text']) # for running benchmark
bench_labels = list(bench_df['answer'])

##### Benchmark functions

In [84]:
multifin_prompt_instruction = """In this task, you're working with English headlines. 
                        This dataset is made up of real-world article headlines from a large accounting firm's websites. 
                        Your objective is to categorize each headline as one of the predefined categories.
                        Categories: ['Finance', 'Technology', 'Tax & Accounting', 'Business & Management', 
                        'Government & Controls', 'Industry']. Reply with one word only. 
                        Following are a few examples of similar text and the correct category.
                        Provide the answer for the last body of text."""

In [24]:
get_prompt

<function __main__.get_prompt(question='', instruction=False, few_shot_instruction=[], few_shot_response=[])>

In [25]:
get_pipe

<function __main__.get_pipe()>

In [26]:
query_model

<function __main__.query_model(model_pipe, question='', instruction=False, few_shot_instruction=[], few_shot_response=[], formatted_response=True)>

In [27]:
format_response

<function __main__.format_response(output)>

In [89]:
'''
at this point I am not renaming the function, because I have ran it so many times.
However, it should really just be called "run_custom_benchmark", since it can be ran on multifin or fin_phrasebank datasets.
'''
def run_multifin_benchmark(few_shot_instruction, few_shot_response,
                          bench_questions, bench_labels, instruction = False):
    
    ###### for multifin, but could really apply to any dataset
    
    # set pipeline
    model_pipe = get_pipe()
    torch.manual_seed(0)
    
    # set a results variable to receive for each question: full sequences, formatted_response, label
    results_multifin = {'samples': []}

    #instruction = multifin_prompt_instruction
    
    print(f"{len(bench_questions)} questions to answer\nNumber completed: ")
    i = 1
    # need to loop through all the benchmark test data
    for ind in range(len(bench_questions)):
    
        question, label = bench_questions[ind], bench_labels[ind]
    
        # within the loop, query the model and receive the model answer
        raw_output, formatted_response = query_model(model_pipe, question, instruction,
                                                     few_shot_instruction, few_shot_response,
                                                     formatted_response = True)
            
        #### check the answer against the label to determine accuracy
        model_answer = formatted_response['model_answer']
            
        # question's accuracy - exact
        if (model_answer.upper() == label.upper()):
            question_acc = 1.0
        else: question_acc = 0.0
        
        # question's relative accuracy - takes into account proximity to the correct answer
        if (instruction == False): # if false, then we are using the standard prompt and 3 categories: negative, neutral, positive
            
            ### there's probably a more concise algorithm, but this will work.
            
            if (model_answer.upper() == 'NEGATIVE'):
                
                if (question_acc == 1.0): question_acc_prox = 1.0
                elif (label.upper() == 'NEUTRAL'): question_acc_prox = 0.5
                else: question_acc_prox = 0.0 # == positive
                
            elif (model_answer.upper() == 'NEUTRAL'):
                
                if (question_acc == 1.0): question_acc_prox = 1.0
                elif (label.upper() == 'NEGATIVE'): question_acc_prox = 0.25 ### not sure if this should be weighted as 0.25? too easy and rewards for 0.5
                else: question_acc_prox = 0.25 # correct answer is positive
                
            elif (model_answer.upper() == 'POSITIVE'):
                
                if (question_acc == 1.0): question_acc_prox = 1.0
                elif (label.upper() == 'NEUTRAL'): question_acc_prox = 0.5
                else: question_acc_prox = 0.0 # correct answer is negative
                
                
            else: question_acc_prox = 0.0 # any other answer is not the right format or one of the three choices
        else: question_acc_prox = 0.0
        
        # add an entry to the results['samples'] for this question - with each accuracy score
        question_results = {'raw_output': raw_output,
                            'formatted_response': formatted_response,
                            'model_answer': model_answer,
                            'correct_answer': label,
                            'accuracy_question': question_acc,
                            'accuracy_proximity_question': question_acc_prox
                           }
        
        results_multifin['samples'].append(question_results)
        
        if (i % 50 == 0): print(i, end = ' ')
        i += 1
                
            
    ###### EXACT accuracy
    # determine overall accuracy for the benchmark
    num_samples = len(results_multifin['samples'])
    acc_list = [res['accuracy_question'] for res in results_multifin['samples']]
    acc_aggregate = 0
    for acc in acc_list:
        acc_aggregate += acc
    model_accuracy = acc_aggregate / num_samples
    
    # save as separate key in the results variable
    results_multifin['benchmark_accuracy_exact'] = model_accuracy
    
    ###### PROXIMITY accuracy
    if (instruction == False): # if false, then we are using the standard prompt and 3 categories: negative, neutral, positive
         # determine overall accuracy for the benchmark
        num_samples = len(results_multifin['samples'])
        acc_list = [res['accuracy_proximity_question'] for res in results_multifin['samples']]
        acc_aggregate = 0
        for acc in acc_list:
            acc_aggregate += acc
        model_accuracy_prox = acc_aggregate / num_samples 
    else: model_accuracy_prox = None
        
    # add the accuracy_proximity score
    results_multifin['benchmark_accuracy_proximate'] = model_accuracy_prox
    
    return(results_multifin)

##### (old) testing functions with small samples -> runs well

In [76]:
train.head()

,question,answer,instruct
4008,According to CEO Matti Perkonoja of the parent...,negative,Instruct: According to CEO Matti Perkonoja of ...
4591,"Earlier today , Geberit 's Finnish rival Upono...",negative,"Instruct: Earlier today , Geberit 's Finnish r..."
4126,Possible personnel reductions concern approxim...,negative,Instruct: Possible personnel reductions concer...
4747,Operating profit decreased to EUR 11.2 mn from...,negative,Instruct: Operating profit decreased to EUR 11...
4440,A survey conducted by Taloustutkimus for Sampo...,negative,Instruct: A survey conducted by Taloustutkimus...


In [100]:
# set pipeline
model_pipe = get_pipe()
torch.manual_seed(0)

Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausa

In [101]:
few_shot_fin_instruction = list(few_shot_fin['Instruction'])
few_shot_fin_response = list(few_shot_fin['Response'])
examples_instruction = list(test['question'])

In [103]:
model

PeftModelForCausalLM(
  (base_model): PeftModelForCausalLM(
    (base_model): GPT2LMHeadModel(
      (transformer): GPT2Model(
        (wte): Embedding(50260, 768)
        (wpe): Embedding(1024, 768)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-11): 12 x GPT2Block(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2Attention(
              (c_attn): Conv1D(nf=2304, nx=768)
              (c_proj): Conv1D(nf=768, nx=768)
              (attn_dropout): Dropout(p=0.1, inplace=False)
              (resid_dropout): Dropout(p=0.1, inplace=False)
            )
            (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (mlp): GPT2MLP(
              (c_fc): Conv1D(nf=3072, nx=768)
              (c_proj): Conv1D(nf=768, nx=3072)
              (act): NewGELUActivation()
              (dropout): Dropout(p=0.1, inplace=False)
            )
          )
        )
        (ln_f): LayerNorm((7

In [102]:
torch.manual_seed(0)
raw_output, formatted_response = query_model(model_pipe, examples_instruction[0], False, few_shot_fin_instruction, few_shot_fin_response,
                                             formatted_response = True) # pipe_ was declared earlier. Global var

AttributeError: 'NoneType' object has no attribute 'shape'

In [75]:
# the question to ask 
examples_instruction_test = list(train['question'])[6:8]
examples_response_test = list(train['answer'])[6:8]
few_shot_instruction_test = list(train['Instruction'])[9:12]
few_shot_response_test = list(train['Response'])[9:12]

KeyError: 'Instruction'

In [85]:
torch.manual_seed(0)
raw_output, formatted_response = query_model(model_pipe, examples_instruction[0], False, few_shot_fin_instruction, few_shot_fin_response,
                                             formatted_response = True) # pipe_ was declared earlier. Global var

In [86]:
print(raw_output[0]['generated_text'])

You are a financial expert that analyzes financial news reports. Each report contains numbers and 
                            qualitative descriptions of a subject's financial performance. 
                            Your job is to classify the sentiment of each report as one of the predefined categories.
                            Categories: ['positive', 'neutral', 'negative']. Reply with one word only.
                            Following are a few examples of similar text and the correct category. 
                            Provide the answer for the last body of text.
Text: {Operating profit in the fourth quarter went down to EUR3m from EUR4 .2 m for the corresponding period of 2009 as it included costs of growth projects .}
Answer: {negative}
Text: {( ADPnews ) - Feb 3 , 2010 - Finland-based steel maker Rautaruukki Oyj ( HEL : RTRKS ) , or Ruukki , said today it slipped to a larger-than-expected pretax loss of EUR 46 million ( USD 64.5 m ) in the fourth quarter of 2009 from

In [87]:
formatted_response

{'full_text': ['Text: {TeliaSonera said about $ 100 million will be invested in the next year in the region to bring mobile coverage to about 90 % of Nepal s population . Choose one of the following words: positive, neutral, negative}',
  'Answer: {neutral}'],
 'model_answer': 'neutral'}

In [98]:
results_multifin = run_multifin_benchmark()

Device set to use cuda:0


In [99]:
results_multifin

{'samples': [{'raw_output': [{'generated_text': "You are a financial expert that analyzes financial news reports. Each report contains numbers and \n                        qualitative descriptions of a subject's financial performance. \n                        Your job is to classify the sentiment of each report as one of the predefined categories.\n                        Categories: ['positive', 'neutral', 'negative']. Reply with one word only.\n                        Following are a few examples of similar text and the correct category. \n                        Provide the answer for the last body of text.\nText: {All other charges were dismissed .}\nAnswer: {neutral}\nText: {Automation makes it possible to conduct several tests simultaneously .}\nAnswer: {neutral}\nText: {`` However , the rapidly increasing costs and the strengthening of the euro were challenging for the whole industry , and they impacted on our results . ''}\nAnswer: {negative}\nText: {Ahlstrom , headquartered 

#### Run all benchmarks

- arc_easy - general
- bbh_zeroshot_causal_judgement - general
- custom multifin - specialized
- custom fin phrasebank test data - specialized

In [16]:
# set up lm_eval task manager and implement
task_manager = lm_eval.tasks.TaskManager()

In [ ]:
os.chdir(pre_results_dir)

##### 1. arc_easy

In [35]:
results_arc = lm_eval.simple_evaluate(
    model = "hf", 
    model_args = {"pretrained": model, 
                  "dtype": "bfloat16", "tokenizer": tokenizer},
    tasks = ['arc_easy'
    ],
    task_manager = task_manager,
    log_samples = True, # set to False
    batch_size = 30,
    #limit = 15, # using the full dataset
    random_seed = 42)

`pretrained` model kwarg is not of type `str`. Many other model arguments may be ignored. Please do not launch via accelerate or use `parallelize=True` if passing an existing model this way.
Passed an already-initialized model through `pretrained`, assuming single-process call to evaluate() or custom distributed integration
Running loglikelihood requests: 100%|██████████| 9501/9501 [12:47<00:00, 12.38it/s]
fatal: not a git repository (or any parent up to mount point /sfs/gpfs)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).


In [36]:
del results_arc['config']

In [37]:
os.chdir(pre_results_dir)
with open('results_arc_easy_json', 'w') as f:
    json.dump(results_arc, f)

##### 2. bbh_causal_judgement

In [38]:
results_bbh = lm_eval.simple_evaluate(
    model = "hf", 
    model_args = {"pretrained": model, 
                  "dtype": "bfloat16", "tokenizer": tokenizer},
    tasks = ['bbh_zeroshot_causal_judgement'
    ],
    task_manager = task_manager,
    log_samples = True, # set to False
    batch_size = 20,
    #limit = 15, # using the full dataset
    random_seed = 42)

`pretrained` model kwarg is not of type `str`. Many other model arguments may be ignored. Please do not launch via accelerate or use `parallelize=True` if passing an existing model this way.
Passed an already-initialized model through `pretrained`, assuming single-process call to evaluate() or custom distributed integration
[Task: bbh_zeroshot_causal_judgement] has_training_docs and has_validation_docs are False, using test_docs as fewshot_docs but this is not recommended.
[Task: bbh_zeroshot_causal_judgement] has_training_docs and has_validation_docs are False, using test_docs as fewshot_docs but this is not recommended.
Running generate_until requests: 100%|██████████| 187/187 [10:25<00:00,  3.35s/it] 
fatal: not a git repository (or any parent up to mount point /sfs/gpfs)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).


In [39]:
del results_bbh['config']

In [40]:
with open('results_bbh_causal_json', 'w') as f:
    json.dump(results_bbh, f)

##### 3. multifin (custom)

In [23]:
run_multifin_benchmark

<function __main__.run_multifin_benchmark(few_shot_instruction, few_shot_response, bench_questions, bench_labels)>

In [51]:
results_multifin = run_multifin_benchmark(few_shot_instruction, few_shot_response, 
                                          bench_questions, bench_labels, multifin_prompt_instruction
                                         )

Device set to use cuda:0


534 questions to answer
Number completed: 


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


50 100 150 200 250 300 350 400 450 500 

In [54]:
results_multifin.keys()

dict_keys(['samples', 'benchmark_accuracy'])

In [55]:
os.chdir(pre_results_dir)

In [56]:
with open('results_multifin', 'w') as f:
    json.dump(results_multifin, f)

##### 4. Run on test data 

In [46]:
few_shot_fin_instruction = list(few_shot_fin['Instruction'])
few_shot_fin_response = list(few_shot_fin['Response'])
bench_questions_fin = list(test['question']) # for running benchmark
bench_labels_fin = list(test['answer'])

In [47]:
# results after one epoch and learning_rate = 0.001
results_finPhrase_testset_pre = run_multifin_benchmark(few_shot_fin_instruction, few_shot_fin_response, 
                                                        bench_questions_fin, bench_labels_fin) # run with default prompt 

Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausa

970 questions to answer
Number completed: 
50 100 150 200 250 300 350 400 450 500 550 600 650 700 750 800 850 900 950 

In [50]:
print(results_finPhrase_testset_pre['benchmark_accuracy_exact'])
results_finPhrase_testset_pre['benchmark_accuracy_proximate']

0.26597938144329897


0.4657216494845361

In [51]:
os.chdir(pre_results_dir)

In [52]:
with open('results_finphraseTest', 'w') as f:
    json.dump(results_finPhrase_testset_pre, f)

#### Look at results and examples for each

##### 1. arc_easy

In [54]:
print("--------------------\nResults:\n", results_arc['results'], "\n")
print("--------------------\nSample 1:\n")
print("Answer Key: ", results_arc['samples']['arc_easy'][0]['doc']['answerKey'])
print("\nArgument: ", results_arc['samples']['arc_easy'][0]['arguments'])
print("\nModel Response: ", results_arc['samples']['arc_easy'][0]['resps'])
print("\nModel accuracy for this question: ", results_arc['samples']['arc_easy'][0]['acc'])

print("\n--------------------\nSample 2:\n")
print("Answer Key: ", results_arc['samples']['arc_easy'][1]['doc']['answerKey'])
print("\nArgument: ", results_arc['samples']['arc_easy'][1]['arguments'])
print("\nModel Response: ", results_arc['samples']['arc_easy'][1]['resps'])
print("\nModel accuracy for this question: ", results_arc['samples']['arc_easy'][1]['acc'])

--------------------
Results:
 {'arc_easy': {'alias': 'arc_easy', 'acc,none': 0.4503367003367003, 'acc_stderr,none': 0.010209047724374148, 'acc_norm,none': 0.4065656565656566, 'acc_norm_stderr,none': 0.010079056419223537}} 

--------------------
Sample 1:

Answer Key:  A

Argument:  [('Question: Which statement best explains why photosynthesis is the foundation of most food webs?\nAnswer:', ' Sunlight is the source of energy for nearly all ecosystems.'), ('Question: Which statement best explains why photosynthesis is the foundation of most food webs?\nAnswer:', ' Most ecosystems are found on land instead of in water.'), ('Question: Which statement best explains why photosynthesis is the foundation of most food webs?\nAnswer:', ' Carbon dioxide is more available than other gases.'), ('Question: Which statement best explains why photosynthesis is the foundation of most food webs?\nAnswer:', ' The producers in all ecosystems are plants.')]

Model Response:  [[(-26.11143684387207, False)],

##### 2. bbh_zeroshot_causal_judgement

In [67]:
print("--------------------\nResults:\n", results_bbh['results'], "\n")
print("--------------------\nSample 1:\n")
print("Answer Key: ", results_bbh['samples']['bbh_zeroshot_causal_judgement'][0]['doc']['target'])
print("\nArgument: ", results_bbh['samples']['bbh_zeroshot_causal_judgement'][0]['arguments'])
print("\nModel Response: ", results_bbh['samples']['bbh_zeroshot_causal_judgement'][0]['resps'])
print("\nModel accuracy for this question: ", results_bbh['samples']['bbh_zeroshot_causal_judgement'][0]['exact_match'])

print("--------------------\nSample 2:\n")
print("Answer Key: ", results_bbh['samples']['bbh_zeroshot_causal_judgement'][1]['doc']['target'])
print("\nArgument: ", results_bbh['samples']['bbh_zeroshot_causal_judgement'][1]['arguments'])
print("\nModel Response: ", results_bbh['samples']['bbh_zeroshot_causal_judgement'][1]['resps'])
print("\nModel accuracy for this question: ", results_bbh['samples']['bbh_zeroshot_causal_judgement'][1]['exact_match'])

--------------------
Results:
 {'bbh_zeroshot_causal_judgement': {'alias': 'bbh_zeroshot_causal_judgement', 'exact_match,strict-match': np.float64(0.0), 'exact_match_stderr,strict-match': 0.0, 'exact_match,flexible-extract': np.float64(0.31016042780748665), 'exact_match_stderr,flexible-extract': 0.033916480026151344}} 

--------------------
Sample 1:

Answer Key:  No

Argument:  [('Answer questions about causal attribution.\n\nQ: How would a typical person answer each of the following questions about causation?\nA machine is set up in such a way that it will short circuit if both the black wire and the red wire touch the battery at the same time. The machine will not short circuit if just one of these wires touches the battery. The black wire is designated as the one that is supposed to touch the battery, while the red wire is supposed to remain in some other part of the machine. One day, the black wire and the red wire both end up touching the battery at the same time. There is a shor

##### 3. multifin - custom!

In [57]:
results_multifin['benchmark_accuracy']

0.09363295880149813

In [58]:
results_multifin['samples'][0]

{'raw_output': [{'generated_text': "In this task, you're working with English headlines. \n                        This dataset is made up of real-world article headlines from a large accounting firm's websites. \n                        Your objective is to categorize each headline as one of the predefined categories.\n                        Categories: ['Finance', 'Technology', 'Tax & Accounting', 'Business & Management', \n                        'Government & Controls', 'Industry']. Reply with one word only. \n                        Following are a few examples of similar text and the correct category.\n                        Provide the answer for the last body of text.\nText: {PwC participated in the opening of the market for trading at the London Stock Exchange}\nAnswer: {Finance}\nText: {Removal of the annual limit of the pension and disability contributions – employment costs are to increase}\nAnswer: {Finance}\nText: {Technology has reduced the tax compliance burden on bus

In [59]:
results_multifin['samples'][15]

{'raw_output': [{'generated_text': "In this task, you're working with English headlines. \n                        This dataset is made up of real-world article headlines from a large accounting firm's websites. \n                        Your objective is to categorize each headline as one of the predefined categories.\n                        Categories: ['Finance', 'Technology', 'Tax & Accounting', 'Business & Management', \n                        'Government & Controls', 'Industry']. Reply with one word only. \n                        Following are a few examples of similar text and the correct category.\n                        Provide the answer for the last body of text.\nText: {PwC participated in the opening of the market for trading at the London Stock Exchange}\nAnswer: {Finance}\nText: {Removal of the annual limit of the pension and disability contributions – employment costs are to increase}\nAnswer: {Finance}\nText: {Technology has reduced the tax compliance burden on bus

##### 4. Test set of financial phrasebank

In [53]:
print(results_finPhrase_testset_pre['benchmark_accuracy_exact'])
results_finPhrase_testset_pre['benchmark_accuracy_proximate']

0.26597938144329897


0.4657216494845361

In [54]:
results_finPhrase_testset_pre['samples'][0]

{'raw_output': [{'generated_text': "You are a financial expert that analyzes financial news reports. Each report contains numbers and \n                                    qualitative descriptions of a subject's financial performance. \n                                    Your job is to classify the sentiment of each report as one of the predefined categories.\n                                    Sentiment categories: ['positive', 'neutral', 'negative']. Reply with one word only.\n                                    Following are a few examples of similar text and the correct category. \n                                    Provide the sentiment category for the last body of text.\nText: {Publishing Sweden 's operating loss was EUR 1.1 mn in Q1 of 2009 , compared to a profit of EUR 0.6 mn a year ago .}\nAnswer: {negative}\nText: {The share capital of Alma Media Corporation (business ID 1944757-4)is EUR 45,031,513.80 and it is divided into 75,052,523 shares .}\nAnswer: {neutral}\nText: {

In [57]:
results_finPhrase_testset_pre['samples'][3]

{'raw_output': [{'generated_text': "You are a financial expert that analyzes financial news reports. Each report contains numbers and \n                                    qualitative descriptions of a subject's financial performance. \n                                    Your job is to classify the sentiment of each report as one of the predefined categories.\n                                    Sentiment categories: ['positive', 'neutral', 'negative']. Reply with one word only.\n                                    Following are a few examples of similar text and the correct category. \n                                    Provide the sentiment category for the last body of text.\nText: {Publishing Sweden 's operating loss was EUR 1.1 mn in Q1 of 2009 , compared to a profit of EUR 0.6 mn a year ago .}\nAnswer: {negative}\nText: {The share capital of Alma Media Corporation (business ID 1944757-4)is EUR 45,031,513.80 and it is divided into 75,052,523 shares .}\nAnswer: {neutral}\nText: {

## Step 3: Train the Model


### Prepare for prompt tuning - from HW 6

#### instruct column already added to the training and testing data

#### Prepare model and tokenize data

In [29]:
print(train.shape)
print(test.shape)

(3867, 3)
(970, 3)


In [30]:
train = Dataset.from_pandas(train)
val = Dataset.from_pandas(test)
train = train.map(lambda samples: tokenizer(samples['instruct']), batched = True)
val = val.map(lambda samples: tokenizer(samples['instruct']), batched = True)

Map: 100%|██████████| 970/970 [00:00<00:00, 7611.11 examples/s]


In [31]:
# very poorly formatted response from the model
formatted_prompt = f"Q: {train[0]['question']}\n\nA: "
inputs = tokenizer.encode(formatted_prompt, return_tensors = "pt").to(model.device)
output = model.generate(inputs, max_new_tokens = 150, pad_token_id = tokenizer.pad_token_id, do_sample = False)
generated_text = tokenizer.decode(output[0], skip_special_tokens = True)
print(generated_text)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


KeyboardInterrupt: 

#### PromptTuning Config

In [31]:
from peft import get_peft_model, PromptTuningConfig, TaskType, PromptTuningInit
generation_config = PromptTuningConfig(
    task_type = TaskType.CAUSAL_LM, # task type
    prompt_tuning_init = PromptTuningInit.RANDOM, # init virtual tokens
    num_virtual_tokens = 10,
    tokenizer_name_or_path = "openai-community/gpt2-xl")
model = get_peft_model(model, generation_config)
model.print_trainable_parameters()

trainable params: 7,680 || all params: 124,449,792 || trainable%: 0.0062


### ATTEMPT 1: Train - 1 epoch, learning_rate = 0.001

#### Prepare trainer

In [32]:
##### start with 1 epoch
from transformers import TrainingArguments
def create_training_arguments(path, learning_rate = 0.001, epochs = 1, eval_steps = 150):
    training_args = TrainingArguments(
        output_dir = path, 
        auto_find_batch_size = True,
        learning_rate = learning_rate, 
        num_train_epochs = epochs, 
        logging_steps = eval_steps,
        eval_strategy = "steps",
        eval_steps = eval_steps, 
        save_steps = eval_steps, 
        load_best_model_at_end = True)
    return training_args

In [33]:
trained_model_dir

'/sfs/gpfs/tardis/home/dcc7qe/LLM_project/gpt2_instruct/trained_models'

In [34]:
training_args = create_training_arguments(trained_model_dir)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [35]:
from transformers import Trainer, DataCollatorForLanguageModeling

# creates a trainer object for training
def create_trainer(model, training_args, train_dataset, eval_dataset):
    trainer = Trainer(
        model = model, 
        args = training_args, 
        train_dataset = train_dataset,
        eval_dataset = eval_dataset,
        data_collator = DataCollatorForLanguageModeling(
            tokenizer, 
            mlm = False))
    return trainer

In [36]:
trainer = create_trainer(model, training_args, train, val)

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


#### Run

In [193]:
trainer.train()

Step,Training Loss,Validation Loss
150,2.821000,2.547609
300,2.470700,2.350604
450,2.312200,2.208324


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=484, training_loss=2.5189645782975125, metrics={'train_runtime': 33.6317, 'train_samples_per_second': 114.981, 'train_steps_per_second': 14.391, 'total_flos': 136190306304000.0, 'train_loss': 2.5189645782975125, 'epoch': 1.0})

### ATTEMPT 2: Train - 2 epochs, learning_rate = 0.001

#### Import model

In [67]:
del model
del tokenizer

In [68]:
model_choice = "vicgalle/gpt2-open-instruct-v1" # used throughout the notebook
#model_choice = "meta-llama/Llama-3.2-3B-Instruct"

model, tokenizer = import_model_token(model_choice)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50260, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50260, bias=False)
)


#### Prepare trainer

In [143]:
##### start with 1 epoch
from transformers import TrainingArguments
def create_training_arguments(path, learning_rate = 0.001, epochs = 2, eval_steps = 150):
    training_args = TrainingArguments(
        output_dir = path, 
        auto_find_batch_size = True,
        learning_rate = learning_rate, 
        num_train_epochs = epochs, 
        logging_steps = eval_steps,
        eval_strategy = "steps",
        eval_steps = eval_steps, 
        save_steps = eval_steps, 
        load_best_model_at_end = True)
    return training_args

In [144]:
trained_model_dir

'/sfs/gpfs/tardis/home/dcc7qe/LLM_project/gpt2_instruct/trained_models'

In [145]:
training_args = create_training_arguments(trained_model_dir)

In [146]:
from transformers import Trainer, DataCollatorForLanguageModeling

# creates a trainer object for training
def create_trainer(model, training_args, train_dataset, eval_dataset):
    trainer = Trainer(
        model = model, 
        args = training_args, 
        train_dataset = train_dataset,
        eval_dataset = eval_dataset,
        data_collator = DataCollatorForLanguageModeling(
            tokenizer, 
            mlm = False))
    return trainer

In [147]:
trainer = create_trainer(model, training_args, train, val)

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


#### Run

In [148]:
trainer.train()

Step,Training Loss,Validation Loss
150,2.849400,2.603967
300,2.547000,2.460616
450,2.442300,2.348077
600,1.811700,2.388314
750,1.573300,2.321527
900,1.515200,2.279105


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=968, training_loss=2.078559213433384, metrics={'train_runtime': 65.252, 'train_samples_per_second': 118.525, 'train_steps_per_second': 14.835, 'total_flos': 272298958848000.0, 'train_loss': 2.078559213433384, 'epoch': 2.0})

### ATTEMPT 3: Train - 1 epochs, learning_rate = 0.0001

#### Import model

In [35]:
del model
del tokenizer

In [36]:
del trainer

In [37]:
model_choice = "vicgalle/gpt2-open-instruct-v1" # used throughout the notebook
#model_choice = "meta-llama/Llama-3.2-3B-Instruct"

model, tokenizer = import_model_token(model_choice)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50260, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50260, bias=False)
)


#### Prepare trainer

In [38]:
##### start with 1 epoch
from transformers import TrainingArguments
def create_training_arguments(path, learning_rate = 0.0001, epochs = 1, eval_steps = 150):
    training_args = TrainingArguments(
        output_dir = path, 
        auto_find_batch_size = True,
        learning_rate = learning_rate, 
        num_train_epochs = epochs, 
        logging_steps = eval_steps,
        eval_strategy = "steps",
        eval_steps = eval_steps, 
        save_steps = eval_steps, 
        load_best_model_at_end = True)
    return training_args

In [39]:
trained_model_dir

'/sfs/gpfs/tardis/home/dcc7qe/LLM_project/gpt2_instruct/trained_models'

In [40]:
training_args = create_training_arguments(trained_model_dir)

In [41]:
from transformers import Trainer, DataCollatorForLanguageModeling

# creates a trainer object for training
def create_trainer(model, training_args, train_dataset, eval_dataset):
    trainer = Trainer(
        model = model, 
        args = training_args, 
        train_dataset = train_dataset,
        eval_dataset = eval_dataset,
        data_collator = DataCollatorForLanguageModeling(
            tokenizer, 
            mlm = False))
    return trainer

In [42]:
trainer = create_trainer(model, training_args, train, val)

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


#### Run

In [43]:
trainer.train() # today

Step,Training Loss,Validation Loss
150,3.276600,3.516257
300,2.981600,3.401589
450,2.894900,3.421430


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=484, training_loss=3.037598759674829, metrics={'train_runtime': 29.3164, 'train_samples_per_second': 131.906, 'train_steps_per_second': 16.51, 'total_flos': 113528836224000.0, 'train_loss': 3.037598759674829, 'epoch': 1.0})

In [190]:
trainer.train() # yesterday

Step,Training Loss,Validation Loss
150,2.463700,2.266076
300,2.218400,2.163368
450,2.159300,2.123967


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=484, training_loss=2.273552082786875, metrics={'train_runtime': 33.5104, 'train_samples_per_second': 115.397, 'train_steps_per_second': 14.443, 'total_flos': 135821333376000.0, 'train_loss': 2.273552082786875, 'epoch': 1.0})

#### SAVE MODEL - good one but not the best anymore

In [202]:
formatted_prompt = f"Q: {train[0]['question']}\n\nA: "
inputs = tokenizer.encode(formatted_prompt, return_tensors = "pt").to(model.device)
output = model.generate(inputs, max_new_tokens = 150, pad_token_id = tokenizer.pad_token_id, do_sample = False)
generated_text = tokenizer.decode(output[0], skip_special_tokens = True)
print(generated_text)

Q: According to CEO Matti Perkonoja of the parent company HKScan , the company 's performance in the first quarter of 2010 has remained clearly below the level of the corresponding period in 2009 . Choose one of the following words: positive, neutral, negative

A:  positive, neutral, negative

Response: negative

Response: negative

Response: negative

Response: negative

Response: negative

Response: negative

Response: negative

Response: negative

Response: negative

Response: negative

Response: negative

Response: negative

Response: negative

Response: negative

Response: negative

Response: negative

Response: negative

Response: negative

Response: negative

Response: negative

Response: negative

Response: negative

Response: negative

Response: negative

Response: negative
Response: negative

Response: negative

Response: negative

Response: negative



In [53]:
best_model_results_dir

'/sfs/gpfs/tardis/home/dcc7qe/LLM_project/gpt2_instruct/trained_models/best_model'

In [54]:
model.save_pretrained(best_model_results_dir) # saved at one point as the best model

### ATTEMPT 4: Try LoRA

#### import model

In [111]:
del model
del tokenizer
del trainer

In [112]:
model_choice = "vicgalle/gpt2-open-instruct-v1" # used throughout the notebook
#model_choice = "meta-llama/Llama-3.2-3B-Instruct"

model, tokenizer = import_model_token(model_choice)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50260, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50260, bias=False)
)


#### prep lora

In [113]:
# set up LoraConfig
from peft import get_peft_model, LoraConfig, TaskType
LORA_R = 64
LORA_ALPHA = 64
LORA_DROPOUT = 0.05
lora_config = LoraConfig(
    r = LORA_R,
    lora_alpha = LORA_ALPHA,
    lora_dropout = LORA_DROPOUT,
    bias = "none", 
    task_type = "CAUSAL_LM",
    target_modules = ['c_fc', 
                      'c_proj'], # just use a couple of the mlp layers
)
model = get_peft_model(model, lora_config) 
model.print_trainable_parameters() ### this seems large - surprised that it is larger than llama used in class

trainable params: 7,077,888 || all params: 131,520,000 || trainable%: 5.3816


/scratch/dcc7qe/llm_course/lib/python3.11/site-packages/peft/tuners/lora/layer.py:1264: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [114]:
from transformers import TrainingArguments
##### reference: notebook from lecture

# creates trainer object for training 
def create_training_arguments_lora(path, learning_rate = 0.0001, epochs = 2, eval_steps = 150):
    training_args = TrainingArguments(
        output_dir = path, 
        auto_find_batch_size = True,
        learning_rate = learning_rate,
        num_train_epochs = epochs,
        logging_steps = eval_steps,
        eval_strategy = "steps",
        eval_steps = eval_steps,
        save_steps = eval_steps,
        load_best_model_at_end = True)
    return training_args

In [116]:
training_args = create_training_arguments_lora(trained_model_dir + '/LoRA')

In [117]:
# create trainer (same as what we did in HW6)
from transformers import Trainer, DataCollatorForLanguageModeling

# creates a trainer object for training
def create_trainer_lora(model, training_args, train_dataset, eval_dataset):
    trainer = Trainer(
        model = model, 
        args = training_args, 
        train_dataset = train_dataset,
        eval_dataset = eval_dataset,
        data_collator = DataCollatorForLanguageModeling(
            tokenizer, 
            mlm = False))
    return trainer

In [118]:
trainer = create_trainer_lora(model, training_args, train, val)

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


#### Run LoRA

In [119]:
trainer.train()

Step,Training Loss,Validation Loss
150,3.745500,3.722314
300,3.339100,3.654149
450,3.263000,3.634432
600,3.204600,3.639663
750,3.164900,3.625385
900,3.114000,3.645490


TrainOutput(global_step=968, training_loss=3.287555726106502, metrics={'train_runtime': 53.5761, 'train_samples_per_second': 144.355, 'train_steps_per_second': 18.068, 'total_flos': 246094221542400.0, 'train_loss': 3.287555726106502, 'epoch': 2.0})

### ATTEMPT 5: epochs = 4, learning rate = 0.0001

#### import

In [57]:
del model
del tokenizer
del trainer
model_choice = "vicgalle/gpt2-open-instruct-v1" # used throughout the notebook
model, tokenizer = import_model_token(model_choice)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50260, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50260, bias=False)
)


#### prep trainer

In [63]:
##### start with 1 epoch
from transformers import TrainingArguments
def create_training_arguments(path, learning_rate = 0.0001, epochs = 4, eval_steps = 150):
    training_args = TrainingArguments(
        output_dir = path, 
        auto_find_batch_size = True,
        learning_rate = learning_rate, 
        num_train_epochs = epochs, 
        logging_steps = eval_steps,
        eval_strategy = "steps",
        eval_steps = eval_steps, 
        save_steps = eval_steps, 
        load_best_model_at_end = True)
    return training_args

In [40]:
training_args = create_training_arguments(trained_model_dir)

In [41]:
from transformers import Trainer, DataCollatorForLanguageModeling

# creates a trainer object for training
def create_trainer(model, training_args, train_dataset, eval_dataset):
    trainer = Trainer(
        model = model, 
        args = training_args, 
        train_dataset = train_dataset,
        eval_dataset = eval_dataset,
        data_collator = DataCollatorForLanguageModeling(
            tokenizer, 
            mlm = False))
    return trainer

In [42]:
trainer = create_trainer(model, training_args, train, val)

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


#### Run - rate = 0.0001, 4 epochs -> 2nd best model

In [67]:
trainer.train() 

Step,Training Loss,Validation Loss
150,3.241900,3.523757
300,3.003300,3.476349
450,2.879400,3.475524
600,2.485100,3.557681
750,2.356100,3.515241
900,2.366600,3.577992
1050,2.135500,3.677817
1200,2.036900,3.697888
1350,2.024900,3.747730
1500,1.948700,3.863908


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=1936, training_loss=2.3080819342747207, metrics={'train_runtime': 151.3389, 'train_samples_per_second': 102.208, 'train_steps_per_second': 12.792, 'total_flos': 449023212288000.0, 'train_loss': 2.3080819342747207, 'epoch': 4.0})

In [68]:
few_shot_fin = few_shot_fin.sample(frac = 1)
few_shot_fin_instruction = list(few_shot_fin['Instruction'])
few_shot_fin_response = list(few_shot_fin['Response'])
bench_questions_fin = list(test['question']) # for running benchmark
bench_labels_fin = list(test['answer'])
results_ = run_multifin_benchmark(few_shot_fin_instruction, few_shot_fin_response, 
                                    bench_questions_fin, bench_labels_fin)
print("accuracy-exact:", results_['benchmark_accuracy_exact'], "\n")
print("accuracy-proximate:", results_['benchmark_accuracy_proximate'], "\n")
for i in range(5): print("Model: ", results_['samples'][i]['formatted_response'], "\n", "Correct: ", results_['samples'][i]['correct_answer'], "\n")

Device set to use cuda:0


970 questions to answer
Number completed: 


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


50 100 150 200 250 300 350 400 450 500 550 600 650 700 750 800 850 900 950 accuracy-exact: 0.48556701030927835 

accuracy-proximate: 0.6371134020618556 

Model:  {'full_text': ['Text: {Residents access to the block is planned to be from Aleksandri Street . Choose one of the following words: positive, neutral, negative}', 'Answer: neutral'], 'model_answer': 'neutral'} 
 Correct:  neutral 

Model:  {'full_text': ['Answer: neutral', 'Text'], 'model_answer': 'neutral'} 
 Correct:  neutral 

Model:  {'full_text': ["Text: {The terms of the aforementioned funding are considerably below the Bank 's current CDS levels in the market and have a maturity ranging from 1 to 7.5 years . Choose one of the following words: positive, neutral, negative}", 'Answer: neutral'], 'model_answer': 'neutral'} 
 Correct:  negative 

Model:  {'full_text': ['Text: {Fortum had previously bought the state-held stake in TGK-10 from RAO UES during its reform . Choose one of the following words: positive, neutral, negat

##### 2nd BEST - saved at one point, then overwritten

In [69]:
best_model_results_dir

'/sfs/gpfs/tardis/home/dcc7qe/LLM_project/gpt2_instruct/trained_models/best_model'

In [70]:
model.save_pretrained(best_model_results_dir) # save the best model

### **BEST MODEL - ATTEMPT 6**: rate = 0.0001, epochs = 5

In [58]:
##### start with 1 epoch
from transformers import TrainingArguments
def create_training_arguments(path, learning_rate = 0.0001, epochs = 5, eval_steps = 150):
    training_args = TrainingArguments(
        output_dir = path, 
        auto_find_batch_size = True,
        learning_rate = learning_rate, 
        num_train_epochs = epochs, 
        logging_steps = eval_steps,
        eval_strategy = "steps",
        eval_steps = eval_steps, 
        save_steps = eval_steps, 
        load_best_model_at_end = True)
    return training_args

In [59]:
training_args = create_training_arguments(trained_model_dir)

In [60]:
from transformers import Trainer, DataCollatorForLanguageModeling

# creates a trainer object for training
def create_trainer(model, training_args, train_dataset, eval_dataset):
    trainer = Trainer(
        model = model, 
        args = training_args, 
        train_dataset = train_dataset,
        eval_dataset = eval_dataset,
        data_collator = DataCollatorForLanguageModeling(
            tokenizer, 
            mlm = False))
    return trainer

In [61]:
trainer = create_trainer(model, training_args, train, val)

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


#### **BEST** Run training

In [62]:
trainer.train() 

Step,Training Loss,Validation Loss
150,3.258000,3.539656
300,2.933200,3.429196
450,2.895200,3.363705
600,2.544000,3.540796
750,2.344600,3.573031
900,2.332600,3.582210
1050,2.136300,3.694432
1200,2.024400,3.737149
1350,2.007000,3.735977
1500,1.923500,3.904024


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=2420, training_loss=2.155215175092713, metrics={'train_runtime': 152.6623, 'train_samples_per_second': 126.652, 'train_steps_per_second': 15.852, 'total_flos': 562026912768000.0, 'train_loss': 2.155215175092713, 'epoch': 5.0})

In [63]:
few_shot_fin = few_shot_fin.sample(frac = 1)
few_shot_fin_instruction = list(few_shot_fin['Instruction'])
few_shot_fin_response = list(few_shot_fin['Response'])
bench_questions_fin = list(test['question']) # for running benchmark
bench_labels_fin = list(test['answer'])
results_ = run_multifin_benchmark(few_shot_fin_instruction, few_shot_fin_response, 
                                    bench_questions_fin, bench_labels_fin)
print("\n\naccuracy-exact:", results_['benchmark_accuracy_exact'], "\n")
print("accuracy-proximate:", results_['benchmark_accuracy_proximate'], "\n")
for i in range(5): print("Model: ", results_['samples'][i]['formatted_response'], "\n", "Correct: ", results_['samples'][i]['correct_answer'], "\n")

Device set to use cuda:0


970 questions to answer
Number completed: 
50 100 150 200 250 300 350 400 450 500 550 600 650 700 750 800 850 900 950 

accuracy-exact: 0.5288659793814433 

accuracy-proximate: 0.663659793814433 

Model:  {'full_text': ['Text: {`` BG Crane has been a strong partner for Hiab in Australia for many years . Choose one of the following words: positive, neutral, negative}', 'Answer: neutral'], 'model_answer': 'neutral'} 
 Correct:  neutral 

Model:  {'full_text': ['Text: {The aim is to increase sales by at least one fifth in 2006 . Choose one of the following words: positive, neutral, negative}', 'Answer: neutral'], 'model_answer': 'neutral'} 
 Correct:  positive 

Model:  {'full_text': ['Answer: neutral', 'Text'], 'model_answer': 'neutral'} 
 Correct:  neutral 

Model:  {'full_text': ["Text: {It delivers a different user experience and enables us to widen the market we can address . '' Choose one of the following words: positive, neutral, negative}", 'Answer: neutral'], 'model_answer': 'neu

##### **BEST (final)** Save

In [64]:
best_model_results_dir

'/sfs/gpfs/tardis/home/dcc7qe/LLM_project/gpt2_instruct/trained_models/best_model'

In [65]:
model.save_pretrained(best_model_results_dir) # save the best model

In [66]:
os.chdir(post_results_dir)
with open('results_finphraseTest_postBEST', 'w') as f:
    json.dump(results_, f) # this was post2

In [67]:
results_finphraseTest_post = results_

### Describe results in one paragraph

**ANSWER**

I tried the several combinations of hyperparameters including learning rate, step size, and number of epochs. The best model was attempt 6, with a learning rate of 0.0001 and 5 epochs, because the training loss continued to decrease and the validation loss did not begin to increase significantly until right before the end of the 5th epoch. I noticed early on in training that when I used a learning rate of 0.001, after just a couple epochs the validation loss increased significantly and the model output was not great. I attempted training with well over 10 epochs and have mixed results. Sometimes the training loss decreased much more slowly than with less epochs, but other times I my financial phrasebank benchmark did achieve better scores using between 10-20 epochs. The best model ended up simply using 5 epochs with a moderately low learning rate - 0.0001, and this provided improved performance on the financial phrasebank test data.


## Step 4: Assess Post-training Benchmark Performance (20 points)
Repeat Step 2 using your trained model. 

#### Run all benchmarks

- arc_easy - general
- bbh_zeroshot_causal_judgement - general
- custom multifin - specialized

In [69]:
# set up lm_eval task manager and implement
task_manager = lm_eval.tasks.TaskManager()

In [70]:
os.chdir(post_results_dir) 

#### 1. arc_easy

In [71]:
results_arc = lm_eval.simple_evaluate(
    model = "hf", 
    model_args = {"pretrained": model, 
                  "dtype": "bfloat16", "tokenizer": tokenizer},
    tasks = ['arc_easy'
    ],
    task_manager = task_manager,
    log_samples = True, # set to False
    batch_size = 30,
    #limit = 15, # using the full dataset
    random_seed = 42)

`pretrained` model kwarg is not of type `str`. Many other model arguments may be ignored. Please do not launch via accelerate or use `parallelize=True` if passing an existing model this way.
Passed an already-initialized model through `pretrained`, assuming single-process call to evaluate() or custom distributed integration
Running loglikelihood requests: 100%|██████████| 9501/9501 [00:07<00:00, 1268.73it/s]
fatal: not a git repository (or any parent up to mount point /sfs/gpfs)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).


In [72]:
del results_arc['config']

In [73]:
with open('results_arc_easy_json_post', 'w') as f:
    json.dump(results_arc, f)

#### 2. bbh

In [75]:
results_bbh = lm_eval.simple_evaluate(
    model = "hf", 
    model_args = {"pretrained": model, 
                  "dtype": "bfloat16", "tokenizer": tokenizer},
    tasks = ['bbh_zeroshot_causal_judgement'
    ],
    task_manager = task_manager,
    log_samples = True, # set to False
    batch_size = 20,
    #limit = 15, # using the full dataset
    random_seed = 42)

`pretrained` model kwarg is not of type `str`. Many other model arguments may be ignored. Please do not launch via accelerate or use `parallelize=True` if passing an existing model this way.
Passed an already-initialized model through `pretrained`, assuming single-process call to evaluate() or custom distributed integration
[Task: bbh_zeroshot_causal_judgement] has_training_docs and has_validation_docs are False, using test_docs as fewshot_docs but this is not recommended.
[Task: bbh_zeroshot_causal_judgement] has_training_docs and has_validation_docs are False, using test_docs as fewshot_docs but this is not recommended.
Running generate_until requests: 100%|██████████| 187/187 [00:33<00:00,  5.59it/s]
fatal: not a git repository (or any parent up to mount point /sfs/gpfs)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).


In [76]:
del results_bbh['config']

In [77]:
with open('results_bbh_causal_json_post', 'w') as f:
    json.dump(results_bbh, f)

#### 3. Multifin 

In [90]:
results_multifin_post = run_multifin_benchmark(few_shot_instruction, few_shot_response, 
                                          bench_questions, bench_labels, multifin_prompt_instruction
                                         )

Device set to use cuda:0


534 questions to answer
Number completed: 
50 100 150 200 250 300 350 400 450 500 

In [91]:
with open('results_multifin_post', 'w') as f:
    json.dump(results_multifin_post, f)

#### 4. Fin phrase - 1 epoch, learning = 0.0001 with proper string formatting.

##### variable setup

In [ ]:
few_shot_fin = few_shot_fin.sample(frac = 1)
few_shot_fin_instruction = list(few_shot_fin['Instruction'])
few_shot_fin_response = list(few_shot_fin['Response'])
bench_questions_fin = list(test['question']) # for running benchmark
bench_labels_fin = list(test['answer'])
for l in [few_shot_fin_instruction, few_shot_fin_response, bench_questions_fin, bench_labels_fin]:
    print(len(l))

9
9
970
970


In [47]:
few_shot_fin_response

['positive',
 'neutral',
 'negative',
 'negative',
 'positive',
 'positive',
 'neutral',
 'neutral',
 'negative']

##### **BEST RESULTS**

In [ ]:
###### ran above in training section

In [ ]:
#### after 5 epochs and rate = 0.0001
results_finphraseTest_post = run_multifin_benchmark(few_shot_fin_instruction, few_shot_fin_response, 
                                                        bench_questions_fin, bench_labels_fin)

In [68]:
print("accuracy-exact:", results_['benchmark_accuracy_exact'], "\n")
print("accuracy-proximate:", results_['benchmark_accuracy_proximate'], "\n")
for i in range(5): print("Model: ", results_['samples'][i]['formatted_response'], "\n", "Correct: ", results_['samples'][i]['correct_answer'], "\n")

accuracy-exact: 0.5288659793814433 

accuracy-proximate: 0.663659793814433 

Model:  {'full_text': ['Text: {`` BG Crane has been a strong partner for Hiab in Australia for many years . Choose one of the following words: positive, neutral, negative}', 'Answer: neutral'], 'model_answer': 'neutral'} 
 Correct:  neutral 

Model:  {'full_text': ['Text: {The aim is to increase sales by at least one fifth in 2006 . Choose one of the following words: positive, neutral, negative}', 'Answer: neutral'], 'model_answer': 'neutral'} 
 Correct:  positive 

Model:  {'full_text': ['Answer: neutral', 'Text'], 'model_answer': 'neutral'} 
 Correct:  neutral 

Model:  {'full_text': ["Text: {It delivers a different user experience and enables us to widen the market we can address . '' Choose one of the following words: positive, neutral, negative}", 'Answer: neutral'], 'model_answer': 'neutral'} 
 Correct:  positive 

Model:  {'full_text': ['Answer: neutral', 'Text'], 'model_answer': 'neutral'} 
 Correct:  

In [ ]:
#### results already output in training section
#os.chdir(post_results_dir)
#with open('results_finphraseTest_post', 'w') as f:
#    json.dump(results_finPhrase_testset_post, f) # this was post2

##### LoRA - didn't work for some attempts. Poor format of responses.

In [120]:
##### after LoRA - 2 epochs
results_finPhrase_testset_postLoRA2 = run_multifin_benchmark(few_shot_fin_instruction, few_shot_fin_response, 
                                                        bench_questions_fin, bench_labels_fin)

Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausa

970 questions to answer
Number completed: 
50 100 150 200 250 300 350 400 450 500 550 600 650 700 750 800 850 900 950 

In [122]:
results_finPhrase_testset_postLoRA2['benchmark_accuracy']

0.5154639175257731

In [66]:
######### results: LoRA - after 2 epoch and learning_rate = 0.0001
results_finPhrase_testset_postLoRA = run_multifin_benchmark(few_shot_fin_instruction, few_shot_fin_response, 
                                                        bench_questions_fin, bench_labels_fin)

Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausa

970 questions to answer
Number completed: 
['', '{neutral}']
["'neutral'"]
['', 'Choose', 'one', 'of']
['', '']


IndexError: list index out of range

### Inspect results

#### 1. arc_easy

In [74]:
print("--------------------\nResults:\n", results_arc['results'], "\n")
print("--------------------\nSample 1:\n")
print("Answer Key: ", results_arc['samples']['arc_easy'][0]['doc']['answerKey'])
print("\nArgument: ", results_arc['samples']['arc_easy'][0]['arguments'])
print("\nModel Response: ", results_arc['samples']['arc_easy'][0]['resps'])
print("\nModel accuracy for this question: ", results_arc['samples']['arc_easy'][0]['acc'])

print("\n--------------------\nSample 2:\n")
print("Answer Key: ", results_arc['samples']['arc_easy'][1]['doc']['answerKey'])
print("\nArgument: ", results_arc['samples']['arc_easy'][1]['arguments'])
print("\nModel Response: ", results_arc['samples']['arc_easy'][1]['resps'])
print("\nModel accuracy for this question: ", results_arc['samples']['arc_easy'][1]['acc'])

--------------------
Results:
 {'arc_easy': {'alias': 'arc_easy', 'acc,none': 0.4019360269360269, 'acc_stderr,none': 0.010060521220920566, 'acc_norm,none': 0.38257575757575757, 'acc_norm_stderr,none': 0.00997283779053148}} 

--------------------
Sample 1:

Answer Key:  A

Argument:  [('Question: Which statement best explains why photosynthesis is the foundation of most food webs?\nAnswer:', ' Sunlight is the source of energy for nearly all ecosystems.'), ('Question: Which statement best explains why photosynthesis is the foundation of most food webs?\nAnswer:', ' Most ecosystems are found on land instead of in water.'), ('Question: Which statement best explains why photosynthesis is the foundation of most food webs?\nAnswer:', ' Carbon dioxide is more available than other gases.'), ('Question: Which statement best explains why photosynthesis is the foundation of most food webs?\nAnswer:', ' The producers in all ecosystems are plants.')]

Model Response:  [[(-38.96726989746094, False)],

#### 2. bbh

In [78]:
print("--------------------\nResults:\n", results_bbh['results'], "\n")
print("--------------------\nSample 1:\n")
print("Answer Key: ", results_bbh['samples']['bbh_zeroshot_causal_judgement'][0]['doc']['target'])
print("\nArgument: ", results_bbh['samples']['bbh_zeroshot_causal_judgement'][0]['arguments'])
print("\nModel Response: ", results_bbh['samples']['bbh_zeroshot_causal_judgement'][0]['resps'])
print("\nModel accuracy for this question: ", results_bbh['samples']['bbh_zeroshot_causal_judgement'][0]['exact_match'])

print("--------------------\nSample 2:\n")
print("Answer Key: ", results_bbh['samples']['bbh_zeroshot_causal_judgement'][1]['doc']['target'])
print("\nArgument: ", results_bbh['samples']['bbh_zeroshot_causal_judgement'][1]['arguments'])
print("\nModel Response: ", results_bbh['samples']['bbh_zeroshot_causal_judgement'][1]['resps'])
print("\nModel accuracy for this question: ", results_bbh['samples']['bbh_zeroshot_causal_judgement'][1]['exact_match'])

--------------------
Results:
 {'bbh_zeroshot_causal_judgement': {'alias': 'bbh_zeroshot_causal_judgement', 'exact_match,strict-match': np.float64(0.0), 'exact_match_stderr,strict-match': 0.0, 'exact_match,flexible-extract': np.float64(0.2727272727272727), 'exact_match_stderr,flexible-extract': 0.032655509459413576}} 

--------------------
Sample 1:

Answer Key:  No

Argument:  [('Answer questions about causal attribution.\n\nQ: How would a typical person answer each of the following questions about causation?\nA machine is set up in such a way that it will short circuit if both the black wire and the red wire touch the battery at the same time. The machine will not short circuit if just one of these wires touches the battery. The black wire is designated as the one that is supposed to touch the battery, while the red wire is supposed to remain in some other part of the machine. One day, the black wire and the red wire both end up touching the battery at the same time. There is a short

#### 3. Multifin

In [93]:
results_multifin_post['benchmark_accuracy_exact']

0.06928838951310862

#### 4. Fin Phrasebank test data

In [ ]:
#### ran in sections above - copied over

In [68]:
print("accuracy-exact:", results_['benchmark_accuracy_exact'], "\n")
print("accuracy-proximate:", results_['benchmark_accuracy_proximate'], "\n")
for i in range(5): print("Model: ", results_['samples'][i]['formatted_response'], "\n", "Correct: ", results_['samples'][i]['correct_answer'], "\n")

accuracy-exact: 0.5288659793814433 

accuracy-proximate: 0.663659793814433 

Model:  {'full_text': ['Text: {`` BG Crane has been a strong partner for Hiab in Australia for many years . Choose one of the following words: positive, neutral, negative}', 'Answer: neutral'], 'model_answer': 'neutral'} 
 Correct:  neutral 

Model:  {'full_text': ['Text: {The aim is to increase sales by at least one fifth in 2006 . Choose one of the following words: positive, neutral, negative}', 'Answer: neutral'], 'model_answer': 'neutral'} 
 Correct:  positive 

Model:  {'full_text': ['Answer: neutral', 'Text'], 'model_answer': 'neutral'} 
 Correct:  neutral 

Model:  {'full_text': ["Text: {It delivers a different user experience and enables us to widen the market we can address . '' Choose one of the following words: positive, neutral, negative}", 'Answer: neutral'], 'model_answer': 'neutral'} 
 Correct:  positive 

Model:  {'full_text': ['Answer: neutral', 'Text'], 'model_answer': 'neutral'} 
 Correct:  

## Step 5: Interpretation of Results (20 points)
In one-two paragraphs, summarize the results from training your model, noting how the outputs improved post training, how performance on the benchmarks and testing split changed post training, and whether and how the improvements and drawbacks from training you noticed empirically matched those you anticipated in Step 1 above. 

**ANSWER**

The prompt tuning of GPT2-instruct, combined with few shot prompting and prompt engineering, was moderately effective at enhancing the model's ability to classify financial news sentiment, using the 'best model' described above. The primary benchmark I used to assess this was the test set from the financial phrasebank dataset, over 900 rows. I created two metrics in the benchmark using this test data. One determined the model's accuracy in selecting the exact answer, and a second metric computed the model answer's proximity to the correct answer, which punished selecting the category on the opposite side of the scale - choosing positive if the correct answer is negative, and vice versa. Before prompt tuning, the exact-match accuracy was 0.25 and the proximity-accuracy was 0.46. Afterwards, the exact match increased to just over 50%, at 0.52, and the approximate accuracy increased 0.66, indicating that model was moderately effective at choosing close to the right answer. If we were to allow this sentiment classification to feed financial decisionmaking, after prompt tuning, we will have at least largely prevented the model from advising completely opposite of the right decision. 

I originally intended to use multifin as the specialized benchmark. It performed very poorly before prompt tuning and even slightly worse afterwards, probably because it had too many categories, so I decided to focus on the financial phrasebank test dataset as the primary benchmark. The two generalized benchmarks did not rate an accuracy much lower afterwards, roughly 5% lower for each metric, although when looking at the output it was clearly less coherent, resulting from some forgetting. Because the primary benchmark performed much better, I am okay with this. I only intend to use the model for financial sentiment analysis. That said, while I saw significant improvement in the model, increasing to above 50%, it is not ready to be deployed and feed decision making. The GPT2 instruct model, even with few shot prompt engineering, did not provide concise responses and required a function with a formatted one-word response. With more computing resources I would want to run the same model training approach, prompt engineering, benchmarks, and financial dataset on Llama 3-3B instruct. I intended to use Llama initially, although I struggled to find the computing resources needed and switched to GPT2. Because the training dataset included slightly under 4,000 rows, which should be large enough, it is worth exploring whether there is a second labeled dataset, with three sentiment categories but not necessarily finance-related, that could be added on to the financial phrasebank data for training.